<a href="https://colab.research.google.com/github/kwojtylko/flat_price_estimator/blob/main/1_web_scraping_%26_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Predykcja ceny 1 m2 mieszkania  na rynku pierwotym w Krakowie na podstawie ofert zamieszczonych na portalu gratka.pl 

In [1]:
# import bibliotek
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

# Pobranie danych

In [ ]:
def get_offers_gratka(real_estate="mieszkania", city="krakow", market="pierwotny"):

  # tworzenie listy z nazwami cech
  features  = [ ["url_base", 
                "url_page", 
                "page",
                "location",  
                "title", 
                "link",
                "description",
                "area_m2", 
                "n_rooms",
                "n_floor", 
                "price_per_m2" 
             ] ]


  # pobranie liczby dostępnych stron w wyniku wyszukiwania
  
  URL_base = "https://gratka.pl/nieruchomosci/" + real_estate + "/" + city + "/" + market
  req_url_base = requests.get(URL_base)
  html_url_base = req_url_base.content
  soup_url_base = BeautifulSoup(html_url_base, "lxml")
  n_pages = 110
  


  # pętla do pobrania wszystkich stron
  for page in range(1, n_pages+1):
    time.sleep(1)

    # definiowanie żądania
    URL_page = URL_base + "?page=" + str(page)
    req_page = requests.get(URL_page) 
    print("Strona:", page, "|", "Status:", req_page, sep=" ", end="\n")

    # pobranie strony html
    html_page = req_page.content

    # tworzenie obiektu BeautifulSoup
    soup_page = BeautifulSoup(html_page, "lxml")

    # lista z informacjami o ofertach ze sparsowanej strony
    offers = soup_page.find_all("article", {"class": "teaserUnified"})
    print("Lista ofert na stronie:", len(offers), "| Strona: [", page, "/", n_pages, "]", sep=" ", end="\n")

    for offer in offers:

      # pobranie lokalizacji oferty
      location = offer.find("span", {"class": "teaserUnified__location"}).text.strip()

      # pobranie tytułu oferty
      title = offer.find("a", {"class": "teaserUnified__anchor"}).text.strip()

      # pobranie linku do oferty
      link = offer.find("a").get("href")

      # pobranie opisu oferty
      description = np.NaN if offer.find("p", {"class": "teaserUnified__description"})==None else offer.find("p", {"class": "teaserUnified__description"}).text.strip()

      # pobranie ceny zł/m2 z oferty
      price_per_m2 = np.NaN if offer.find("span", {"class": "teaserUnified__additionalPrice"})==None else offer.find("span", {"class": "teaserUnified__additionalPrice"}).text.strip("zł/m²").strip("\n").replace(" ","").replace(",",".") 

      # pobranie informacji o metrazu, liczbie pokoi oraz nr piętra
      offer_details = offer.find_all("li", {"class": "teaserUnified__listItem"})
      
      # tworzenie pętli dla uzupełnienia listy z dodatkową informacją
      # o metrazu, liczbie pokoi oraz nr piętra
      offer_detail_list = []
      for offer_detail in offer_details:
          offer_detail_list.append(offer_detail.text)
          #print(offer_detail_list)
      
      # pobranie metraza
      area_m2 = "".join(list(filter(lambda i: "m2" in i, offer_detail_list))).replace(" m2", "").replace(",",".")

      # pobranie liczby pokoi
      n_rooms = "".join(list(filter(lambda i: " pok" in i, offer_detail_list))).replace(" pokoje", "").replace(" pokój", "").replace(" pokoi", "")
      
      # pobranie nr piętra       
      n_floor = 0 if "parter" in offer_detail_list else "".join(list(filter(lambda i: "piętro" in i, offer_detail_list))).replace(" piętro", "")



      features.append([URL_base, 
                URL_page, 
                page,
                location,  
                title, 
                link,
                description,
                area_m2, 
                n_rooms,
                n_floor, 
                price_per_m2 
             ])
      
  # przekształcenie zbioru danych w DF 
  return(pd.DataFrame(features[1:], columns=features[0]))

In [ ]:
# przygotowanie df
gratka_df = get_offers_gratka()

Strona: 1 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 1 / 110 ]
Strona: 2 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 2 / 110 ]
Strona: 3 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 3 / 110 ]
Strona: 4 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 4 / 110 ]
Strona: 5 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 5 / 110 ]
Strona: 6 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 6 / 110 ]
Strona: 7 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 7 / 110 ]
Strona: 8 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 8 / 110 ]
Strona: 9 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 9 / 110 ]
Strona: 10 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 10 / 110 ]
Strona: 11 | Status: <Response [200]>
Lista ofert na stronie: 32 | Strona: [ 11 / 110 ]
Strona: 12 | Status: <Response [200]>
Lista ofert 

In [ ]:
# przegląd danych
gratka_df

,url_base,url_page,page,location,title,link,description,area_m2,n_rooms,n_floor,price_per_m2
0,https://gratka.pl/nieruchomosci/mieszkania/kra...,https://gratka.pl/nieruchomosci/mieszkania/kra...,1,"Kraków, ...","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkani...,Poznajcie bliżej naszą inwestycję – Krowodrza ...,44.93,2,7,10700zł/m2
1,https://gratka.pl/nieruchomosci/mieszkania/kra...,https://gratka.pl/nieruchomosci/mieszkania/kra...,1,"Kraków, ...","Nowe mieszkanie Kraków Podgórze, ul. Golikówka",https://gratka.pl/nieruchomosci/nowe-mieszkani...,Kukla Nieruchomości prezentujeMieszkanie 2- po...,42,2,1,7024zł/m2
2,https://gratka.pl/nieruchomosci/mieszkania/kra...,https://gratka.pl/nieruchomosci/mieszkania/kra...,1,"Kraków, ...","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkani...,Poznajcie bliżej naszą inwestycję – Krowodrza ...,59.15,3,1,9400zł/m2
3,https://gratka.pl/nieruchomosci/mieszkania/kra...,https://gratka.pl/nieruchomosci/mieszkania/kra...,1,"Kraków, ...","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkani...,Poznajcie bliżej naszą inwestycję – Krowodrza ...,50.56,2,7,10300zł/m2
4,https://gratka.pl/nieruchomosci/mieszkania/kra...,https://gratka.pl/nieruchomosci/mieszkania/kra...,1,"Kraków, ...","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkani...,Poznajcie bliżej naszą inwestycję – Krowodrza ...,76.35,4,6,9700zł/m2
...,...,...,...,...,...,...,...,...,...,...,...
3515,https://gratka.pl/nieruchomosci/mieszkania/kra...,https://gratka.pl/nieruchomosci/mieszkania/kra...,110,"Kraków, ...","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkani...,W budynkach Myśliwska Solar Garden w Krakowie-...,42.22,2,0,9995zł/m2
3516,https://gratka.pl/nieruchomosci/mieszkania/kra...,https://gratka.pl/nieruchomosci/mieszkania/kra...,110,"Kraków, ...","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkani...,W budynkach Myśliwska Solar Garden w Krakowie-...,67.07,3,1,8916zł/m2
3517,https://gratka.pl/nieruchomosci/mieszkania/kra...,https://gratka.pl/nieruchomosci/mieszkania/kra...,110,"Kraków, ...","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkani...,W budynkach Myśliwska Solar Garden w Krakowie-...,63.05,3,1,9056zł/m2
3518,https://gratka.pl/nieruchomosci/mieszkania/kra...,https://gratka.pl/nieruchomosci/mieszkania/kra...,110,"Kraków, ...","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkani...,W budynkach Myśliwska Solar Garden w Krakowie-...,62.59,3,0,9394zł/m2


In [ ]:
# zapisywanie pliku z danymi
gratka_df.to_csv("gratka_dane.csv", index=False)

## Wstępne oczyszczanie danych

In [ ]:
# wczytywanie danych
gratka_df = pd.read_csv("gratka_dane.csv")

In [ ]:
pd.set_option('max_colwidth', 400)
gratka_df

,url_base,url_page,page,location,title,link,description,area_m2,n_rooms,n_floor,price_per_m2
0,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14399332,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",44.93,2,7.0,10700zł/m2
1,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Podgórze, małopolskie","Nowe mieszkanie Kraków Podgórze, ul. Golikówka",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-podgorze-ul-golikowka/ob/19878365,Kukla Nieruchomości prezentujeMieszkanie 2- pokojowe o powierzchni 42 m2 zlokalizowane w domu wolnostojącym przy ulicy Golikówka w dzielnicy Rybitwy. Mieszkanie,42.00,2,1.0,7024zł/m2
2,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14403264,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",59.15,3,1.0,9400zł/m2
3,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14398368,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",50.56,2,7.0,10300zł/m2
4,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14398296,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",76.35,4,6.0,9700zł/m2
...,...,...,...,...,...,...,...,...,...,...,...
3516,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=110,110,"Kraków, Płaszów, małopolskie","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-plaszow-ul-mysliwska/ob/17790039,W budynkach Myśliwska Solar Garden w Krakowie-Płaszowie na piętrach oraz na poziomie garażu zaplanowane są komórki lokatorskie w cenie 3250 zł/m2. Metraże komór,42.22,2,0.0,9995zł/m2
3517,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=110,110,"Kraków, Płaszów, małopolskie","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-plaszow-ul-mysliwska/ob/17790031,W budynkach Myśliwska Solar Garden w Krakowie-Płaszowie na piętrach oraz na poziomie garażu zaplanowane są komórki lokatorskie w cenie 3250 zł/m2. Metraże komór,67.07,3,1.0,8916zł/m2
3518,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=110,110,"Kraków, Płaszów, małopolskie","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-plaszow-ul-mysliwska/ob/17790023,W budynkach Myśliwska Solar Garden w Krakowie-Płaszowie na piętrach oraz na poziomie garażu zaplanowane są komórki lokat

In [ ]:
#rozmiar df
gratka_df.shape

(3521, 11)

In [ ]:
#podstawowe informacje
gratka_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3521 entries, 0 to 3520
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url_base      3521 non-null   object 
 1   url_page      3521 non-null   object 
 2   page          3521 non-null   int64  
 3   location      3521 non-null   object 
 4   title         3521 non-null   object 
 5   link          3520 non-null   object 
 6   description   3248 non-null   object 
 7   area_m2       3519 non-null   float64
 8   n_rooms       3514 non-null   object 
 9   n_floor       3484 non-null   float64
 10  price_per_m2  3519 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 302.7+ KB


In [ ]:
#sprawdzenie ilości braków
gratka_df.isnull().sum()

url_base          0
url_page          0
page              0
location          0
title             0
link              1
description     273
area_m2           2
n_rooms           7
n_floor          37
price_per_m2      2
dtype: int64

In [ ]:
#uzupełnianie braków/ usuwanie braków
gratka_df["description"].fillna("brak", inplace=True)

In [ ]:
gratka_df.dropna(inplace=True)

In [ ]:
gratka_df.isnull().sum()

url_base        0
url_page        0
page            0
location        0
title           0
link            0
description     0
area_m2         0
n_rooms         0
n_floor         0
price_per_m2    0
dtype: int64

In [ ]:
#zmiana typów danych
gratka_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3483 entries, 0 to 3520
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url_base      3483 non-null   object 
 1   url_page      3483 non-null   object 
 2   page          3483 non-null   int64  
 3   location      3483 non-null   object 
 4   title         3483 non-null   object 
 5   link          3483 non-null   object 
 6   description   3483 non-null   object 
 7   area_m2       3483 non-null   float64
 8   n_rooms       3483 non-null   object 
 9   n_floor       3483 non-null   float64
 10  price_per_m2  3483 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 326.5+ KB


In [ ]:
gratka_df["n_rooms"].value_counts()

3               1245
2               1171
1                529
4                399
5                114
6                 23
więcej niż 8       2
Name: n_rooms, dtype: int64

In [ ]:
more8rooms = gratka_df.query("n_rooms == 'więcej niż 8' ")
more8rooms

,url_base,url_page,page,location,title,link,description,area_m2,n_rooms,n_floor,price_per_m2
3392,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=106,106,"Kraków, Dębniki, małopolskie","Nowe mieszkanie Kraków Dębniki, ul. Tyniecka",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-debniki-ul-tyniecka/ob/18575333,Atrakcyjny apartamentowiec dla inwestoraLOKALIZACJA:Budynek zlokalizowany jest na Podgórzu w okolicy ulicy Tynieckiej i Widłakowej. Niewątpliwą zaletą,365.0,więcej niż 8,1.0,5342zł/m2
3403,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=107,107,"Kraków, Stradom, małopolskie","Nowe mieszkanie Kraków Stradom, ul. Koletek 12",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-stradom-ul-koletek-12/ob/18488123,"Do sprzedaży piękny apartament królewski, położony w luksusowym kompleksie Angel Wawel tuż przy Bulwarach Wiślanych, Zamku Wawel oraz modnej dzielnicy Kazimierz",500.0,więcej niż 8,0.0,30000zł/m2


In [ ]:
gratka_df.drop(index=[3392,3403], inplace=True)

In [ ]:
#sprawdzenie
gratka_df["n_rooms"].value_counts()

3    1245
2    1171
1     529
4     399
5     114
6      23
Name: n_rooms, dtype: int64

In [ ]:
gratka_df["n_rooms"] = gratka_df["n_rooms"].astype(int)

In [ ]:
gratka_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3481 entries, 0 to 3520
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url_base      3481 non-null   object 
 1   url_page      3481 non-null   object 
 2   page          3481 non-null   int64  
 3   location      3481 non-null   object 
 4   title         3481 non-null   object 
 5   link          3481 non-null   object 
 6   description   3481 non-null   object 
 7   area_m2       3481 non-null   float64
 8   n_rooms       3481 non-null   int64  
 9   n_floor       3481 non-null   float64
 10  price_per_m2  3481 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 326.3+ KB


In [ ]:

gratka_df["price_per_m2"] = gratka_df["price_per_m2"].str.replace("zł/m2", "").astype(int)

In [ ]:
#df po przekształceniach
gratka_df.reset_index(drop=True, inplace=True)
gratka_df

,url_base,url_page,page,location,title,link,description,area_m2,n_rooms,n_floor,price_per_m2
0,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14399332,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",44.93,2,7.0,10700
1,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Podgórze, małopolskie","Nowe mieszkanie Kraków Podgórze, ul. Golikówka",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-podgorze-ul-golikowka/ob/19878365,Kukla Nieruchomości prezentujeMieszkanie 2- pokojowe o powierzchni 42 m2 zlokalizowane w domu wolnostojącym przy ulicy Golikówka w dzielnicy Rybitwy. Mieszkanie,42.00,2,1.0,7024
2,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14403264,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",59.15,3,1.0,9400
3,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14398368,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",50.56,2,7.0,10300
4,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14398296,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",76.35,4,6.0,9700
...,...,...,...,...,...,...,...,...,...,...,...
3476,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=110,110,"Kraków, Płaszów, małopolskie","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-plaszow-ul-mysliwska/ob/17790039,W budynkach Myśliwska Solar Garden w Krakowie-Płaszowie na piętrach oraz na poziomie garażu zaplanowane są komórki lokatorskie w cenie 3250 zł/m2. Metraże komór,42.22,2,0.0,9995
3477,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=110,110,"Kraków, Płaszów, małopolskie","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-plaszow-ul-mysliwska/ob/17790031,W budynkach Myśliwska Solar Garden w Krakowie-Płaszowie na piętrach oraz na poziomie garażu zaplanowane są komórki lokatorskie w cenie 3250 zł/m2. Metraże komór,67.07,3,1.0,8916
3478,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=110,110,"Kraków, Płaszów, małopolskie","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-plaszow-ul-mysliwska/ob/17790023,W budynkach Myśliwska Solar Garden w Krakowie-Płaszowie na piętrach oraz na poziomie garażu zaplanowane są komórki lokatorskie w cenie 3250 zł/m2. Metraże 

In [ ]:
#rozmiar df po usunięciu braków
gratka_df.shape

(3481, 11)

In [ ]:
#podstawowe informacje po przekształceniach
gratka_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3481 entries, 0 to 3480
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url_base      3481 non-null   object 
 1   url_page      3481 non-null   object 
 2   page          3481 non-null   int64  
 3   location      3481 non-null   object 
 4   title         3481 non-null   object 
 5   link          3481 non-null   object 
 6   description   3481 non-null   object 
 7   area_m2       3481 non-null   float64
 8   n_rooms       3481 non-null   int64  
 9   n_floor       3481 non-null   float64
 10  price_per_m2  3481 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 299.3+ KB


In [ ]:
#wstępne statystyki danych
gratka_df.describe()

,page,area_m2,n_rooms,n_floor,price_per_m2
count,3481.000000,3481.000000,3481.000000,3481.000000,3481.000000
mean,55.598966,55.448914,2.559609,2.392129,10631.935363
std,31.648347,26.350922,1.027479,2.197552,2941.337028
min,1.000000,13.300000,1.000000,0.000000,4999.000000
25%,28.000000,39.000000,2.000000,1.000000,8893.000000
50%,56.000000,51.080000,3.000000,2.000000,9940.000000
75%,83.000000,66.110000,3.000000,4.000000,11955.000000
max,110.000000,230.000000,6.000000,14.000000,40000.000000


In [ ]:
# zapisywanie pliku z danymi
gratka_df.to_csv("gratka_dane_new.csv", index=False)